In [1]:
import pandas as pd
import numpy as np
import h5py
import os
import glob
import shutil

In [2]:
#convienence functions for .visit() and constants/globals/fixed params

#constants
sep='\n\n-------------------'
nameList=['fluxCo2','fluxMome','fluxTemp','fluxH2o','foot']#this may be added to due to the immense amount of data in
                                                           #each hdf5 file, watch for uniqueness of keys
#fixed params
np.set_printoptions(threshold=500)#2000)

#globals, must watch carefully
fi=None
fd=None
cumulative_null=0
tot_sum=0
 
#convienence functions
def printname(name):#prints names of all groups/subgrups/dataset file names
    print(name)

def attrsearch(name,node):#prints the attributes of all groups/subgrups/dataset file names
    attr = list(node.attrs.items())
    print(name, node, attr)
    print(node.attrs)
    print('\n')
    
def appendto(name,node):#appends to our current generated file
    global fi,fd,searchval#must watch carefully
    if searchval in name and not searchval in node.parent.name:
        fd.copy(node, fd, name=name)
        
def printdat(name,node):#for data inspection of entire file
    if isinstance(node, h5py.Dataset) and not name=='readMe' and not name=='objDesc': 
        print(name, node, list(node.attrs.items()))
        print(node.value)
        nullnumcnt(node)
        print('\n')
        
def nullnumcnt(node):#converts the resulting np arr of values to dataframe, counts nulls and updates the running sums
    global cumulative_null,tot_sum
    if 'data' in node.name:
        df2=pd.DataFrame(node.value)
        nullnum=df2.isnull().values.ravel().sum()
        cumulative_null+=nullnum
        tot=len(df2.index)
        tot_sum+=tot
        print('Nullnum=%s tot=%s percent_null=%s'%(str(nullnum),str(tot),str((float(nullnum)/tot)*100)))
        
def read_process(sourcefilename):#read and process the individual hdf5 files, generated a separated file
    global fi,fd#must watch carefully
    fi=h5py.File(sourcefilename, 'r')
    fd=h5py.File(sourcefilename[:-3]+'_'+nameList[0]+sourcefilename[-3:], 'w')#fd is file destination
    fi.visititems(appendto)#need to modify this
    fd.copy(fi['readMe'], fd, name='readMe')#copy the readMe and objDesc from original file
    fd.copy(fi['objDesc'], fd, name='objDesc')
    fd.visit(printname)
    fd.visititems(printdat)
    fd.close()
    fi.close()

In [3]:
#user variables
searchterm='NEON.D18.BARR'+'*'#hdf5 data file should contain this
foldername='NEON_eddy-flux-2'#folder name with all eddy files in it
searchval=nameList[0]

In [4]:
os.chdir(foldername)

In [5]:
#splits hdf5 and prints structure and logging info
count=0#result should be 12, one for each month
fnametest=''
for fname in glob.glob(searchterm):
    print(fname)
    shutil.unpack_archive(fname,fname[:-4],'zip')
    print('unzipped&extension removed:')
    print(fname[:-4])
    print('contains')
    for subfname in glob.glob(fname[:-4]+"/*.h5"):#search for h5 file in subdirectory we unzipped
        print(subfname)
        fnametest=subfname
        read_process(subfname)
    #break
    print('\n')
    print(sep)
    count+=1
print(count)
print('Nullnum_tot=%s tot_sum=%s tot_percent_null=%s'%(str(cumulative_null),str(tot_sum),str((float(cumulative_null)/tot_sum)*100)))

NEON.D18.BARR.DP4.00200.001.2018-08.basic.20190211T221124Z.zip
unzipped&extension removed:
NEON.D18.BARR.DP4.00200.001.2018-08.basic.20190211T221124Z
contains
NEON.D18.BARR.DP4.00200.001.2018-08.basic.20190211T221124Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-08.basic.h5
BARR
BARR/dp04
BARR/dp04/data
BARR/dp04/data/fluxCo2
BARR/dp04/data/fluxCo2/nsae
BARR/dp04/data/fluxCo2/stor
BARR/dp04/data/fluxCo2/turb
BARR/dp04/qfqm
BARR/dp04/qfqm/fluxCo2
BARR/dp04/qfqm/fluxCo2/stor
BARR/dp04/ucrt
BARR/dp04/ucrt/fluxCo2
objDesc
readMe
BARR/dp04/data/fluxCo2/nsae <HDF5 dataset "nsae": shape (1248,), type "|V58"> [('unit', array([b'umolCo2 m-2 s-1'], dtype='|S16'))]
[(b'2018-08-03T00:00:00.000Z', b'2018-08-03T00:29:59.000Z', nan)
 (b'2018-08-03T00:30:00.000Z', b'2018-08-03T00:59:59.000Z', nan)
 (b'2018-08-03T01:00:00.000Z', b'2018-08-03T01:29:59.000Z', nan) ...
 (b'2018-08-31T22:30:00.000Z', b'2018-08-31T22:59:59.000Z', nan)
 (b'2018-08-31T23:00:00.000Z', b'2018-08-31T23:29:59.000Z', nan)
 (b'2018-08-31T

unzipped&extension removed:
NEON.D18.BARR.DP4.00200.001.2018-02.basic.20190201T165058Z
contains
NEON.D18.BARR.DP4.00200.001.2018-02.basic.20190201T165058Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-02.basic.h5
BARR
BARR/dp04
BARR/dp04/data
BARR/dp04/data/fluxCo2
BARR/dp04/data/fluxCo2/nsae
BARR/dp04/data/fluxCo2/stor
BARR/dp04/data/fluxCo2/turb
BARR/dp04/qfqm
BARR/dp04/qfqm/fluxCo2
BARR/dp04/qfqm/fluxCo2/stor
BARR/dp04/ucrt
BARR/dp04/ucrt/fluxCo2
objDesc
readMe
BARR/dp04/data/fluxCo2/nsae <HDF5 dataset "nsae": shape (864,), type "|V58"> [('unit', array([b'umolCo2 m-2 s-1'], dtype='|S16'))]
[(b'2018-02-01T00:00:00.000Z', b'2018-02-01T00:29:59.000Z', nan)
 (b'2018-02-01T00:30:00.000Z', b'2018-02-01T00:59:59.000Z', nan)
 (b'2018-02-01T01:00:00.000Z', b'2018-02-01T01:29:59.000Z', nan) ...
 (b'2018-02-28T22:30:00.000Z', b'2018-02-28T22:59:59.000Z', nan)
 (b'2018-02-28T23:00:00.000Z', b'2018-02-28T23:29:59.000Z', nan)
 (b'2018-02-28T23:30:00.000Z', b'2018-02-28T23:59:59.000Z', nan)]
Nullnum=484 t

unzipped&extension removed:
NEON.D18.BARR.DP4.00200.001.2018-11.basic.20190307T172445Z
contains
NEON.D18.BARR.DP4.00200.001.2018-11.basic.20190307T172445Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-11.basic.h5
BARR
BARR/dp04
BARR/dp04/data
BARR/dp04/data/fluxCo2
BARR/dp04/data/fluxCo2/nsae
BARR/dp04/data/fluxCo2/stor
BARR/dp04/data/fluxCo2/turb
BARR/dp04/qfqm
BARR/dp04/qfqm/fluxCo2
BARR/dp04/qfqm/fluxCo2/stor
BARR/dp04/ucrt
BARR/dp04/ucrt/fluxCo2
objDesc
readMe
BARR/dp04/data/fluxCo2/nsae <HDF5 dataset "nsae": shape (1392,), type "|V58"> [('unit', array([b'umolCo2 m-2 s-1'], dtype='|S16'))]
[(b'2018-11-02T00:00:00.000Z', b'2018-11-02T00:29:59.000Z',        nan)
 (b'2018-11-02T00:30:00.000Z', b'2018-11-02T00:59:59.000Z', 0.43495508)
 (b'2018-11-02T01:00:00.000Z', b'2018-11-02T01:29:59.000Z', 1.49854074)
 ...
 (b'2018-11-30T22:30:00.000Z', b'2018-11-30T22:59:59.000Z',        nan)
 (b'2018-11-30T23:00:00.000Z', b'2018-11-30T23:29:59.000Z',        nan)
 (b'2018-11-30T23:30:00.000Z', b'2018-11-3

BARR
BARR/dp04
BARR/dp04/data
BARR/dp04/data/fluxCo2
BARR/dp04/data/fluxCo2/nsae
BARR/dp04/data/fluxCo2/stor
BARR/dp04/data/fluxCo2/turb
BARR/dp04/qfqm
BARR/dp04/qfqm/fluxCo2
BARR/dp04/qfqm/fluxCo2/stor
BARR/dp04/ucrt
BARR/dp04/ucrt/fluxCo2
objDesc
readMe
BARR/dp04/data/fluxCo2/nsae <HDF5 dataset "nsae": shape (1440,), type "|V58"> [('unit', array([b'umolCo2 m-2 s-1'], dtype='|S16'))]
[(b'2018-04-01T00:00:00.000Z', b'2018-04-01T00:29:59.000Z', nan)
 (b'2018-04-01T00:30:00.000Z', b'2018-04-01T00:59:59.000Z', nan)
 (b'2018-04-01T01:00:00.000Z', b'2018-04-01T01:29:59.000Z', nan) ...
 (b'2018-04-30T22:30:00.000Z', b'2018-04-30T22:59:59.000Z', nan)
 (b'2018-04-30T23:00:00.000Z', b'2018-04-30T23:29:59.000Z', nan)
 (b'2018-04-30T23:30:00.000Z', b'2018-04-30T23:59:59.000Z', nan)]
Nullnum=1440 tot=1440 percent_null=100.0


BARR/dp04/data/fluxCo2/stor <HDF5 dataset "stor": shape (1440,), type "|V58"> [('unit', array([b'umolCo2 m-2 s-1'], dtype='|S16'))]
[(b'2018-04-01T00:00:00.000Z', b'2018-04-0